In [1]:
import json
from tqdm.autonotebook import tqdm
import pandas as pd
from itertools import combinations
import sys
sys.path.append('src/')
import utils

author_data = json.load(open('../database/sep_clean/author_cited_count.json'))
ref_raw = json.load(open('../database/sep_clean/ent_ref_sep_dict.json'))
article_data = pd.read_csv('../database/sep_clean/ent_ref_info_list.csv')

entry_list = list(ref_raw.keys())
entry_set = set(entry_list)

%load_ext autoreload
%autoreload 2

/var/folders/8c/vvnr64892s54tsnl93ky1xn00000gn/T/ipykernel_85197/141153657.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/Users/junyitao/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
author_list = [
"Timothy Williamson",
"Alvin I. Goldman",
"Alvin Plantinga",
"Ernest Sosa",
"Fred I. Dretske",
"William P. Alston",]

utils.find_author(author_list)

[('epistemology',
  {'Timothy Williamson': 1,
   'Alvin I. Goldman': 6,
   'Alvin Plantinga': 3,
   'Ernest Sosa': 13,
   'Fred I. Dretske': 4,
   'William P. Alston': 7}),
 ('reliabilism',
  {'Timothy Williamson': 2,
   'Alvin I. Goldman': 10,
   'Alvin Plantinga': 1,
   'Ernest Sosa': 7,
   'Fred I. Dretske': 2,
   'William P. Alston': 2}),
 ('knowledge-analysis',
  {'Timothy Williamson': 3,
   'Alvin I. Goldman': 9,
   'Alvin Plantinga': 2,
   'Ernest Sosa': 2,
   'Fred I. Dretske': 4,
   'William P. Alston': 1}),
 ('testimony-episprob',
  {'Timothy Williamson': 2,
   'Alvin I. Goldman': 4,
   'Alvin Plantinga': 1,
   'Ernest Sosa': 5,
   'Fred I. Dretske': 1,
   'William P. Alston': 2}),
 ('perception-episprob',
  {'Timothy Williamson': 1,
   'Alvin I. Goldman': 4,
   'Alvin Plantinga': 1,
   'Ernest Sosa': 4,
   'Fred I. Dretske': 1,
   'William P. Alston': 3}),
 ('contextualism-epistemology',
  {'Timothy Williamson': 3,
   'Alvin I. Goldman': 4,
   'Ernest Sosa': 6,
   'Fred I. D

In [3]:
citation_list = ["Epistemology and Cognition (1986) ['Alvin I. Goldman']",
     "Warrant and Proper Function (1993) ['Alvin Plantinga']"
]
utils.find_article(citation_list)

[('testimony-episprob',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1,
   "Warrant and Proper Function (1993) ['Alvin Plantinga']": 1}),
 ('knowledge-analysis',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1,
   "Warrant and Proper Function (1993) ['Alvin Plantinga']": 1}),
 ('perception-episprob',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1,
   "Warrant and Proper Function (1993) ['Alvin Plantinga']": 1}),
 ('evidence', {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1}),
 ('epistemology-naturalized',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1}),
 ('epistemology-virtue',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1}),
 ('concepts', {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1}),
 ('self-knowledge-externalism',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1}),
 ('epistemology',
  {"Epistemology and Cognition (1986) ['Alvin I. Goldman']": 1}),
 ('justep-in

In [6]:
{'entA':{
    'link_by_article': {
        'entb': ['articleA'],
        'entc': ['articleB', 'articleC']
    },
    'link_by_author': {
        'entb': ['authorA'],
        'entc': ['authorB', 'authorC']
    },
},
'entB': {...}
}

{'entA': {'link_by_article': {'entb': ['articleA'],
   'entc': ['articleB', 'articleC']},
  'link_by_author': {'entb': ['authorA'], 'entc': ['authorB', 'authorC']}},
 'entB': {Ellipsis}}

In [7]:
def add_to_link_dict(link_dict, entA, entB, link_type, link_name):
    # add entB to entA's link_dict
    working_dict = link_dict[entA][f'link_by_{link_type}']
    if entB in working_dict and link_name not in working_dict[entB]:
        working_dict[entB].append(link_name)
    else:
        working_dict[entB] = [link_name]
    
    

In [8]:
all_links = dict()
for entry in entry_list:
    all_links[entry] = {
        'link_by_article': dict(),
        'link_by_author': dict()
    }


for author_name, info in tqdm(author_data.items()):
    cited_ent_list = info['cited_ent_list'].split('; ')
    for entA, entB in combinations(cited_ent_list, 2):
        add_to_link_dict(all_links, entA, entB, 'author', author_name)
        add_to_link_dict(all_links, entB, entA, 'author', author_name)

            
            

  0%|          | 0/76391 [00:00<?, ?it/s]

In [9]:
for article_title in tqdm(article_data['ref_text']):
    subset = article_data[article_data['ref_text'] == article_title]
    if len(subset) < 2:
        continue
    for entA, entB in combinations(subset['cited_by_ent'], 2):
        add_to_link_dict(all_links, entA, entB, 'article', article_title)
        add_to_link_dict(all_links, entB, entA, 'article', article_title)

  0%|          | 0/200328 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
json.dump(all_links, open('../database/sep_clean/ent_link_dict.json', 'w'), indent=4)